# 세미프로젝트 1-2

## 서울시 공공 자전거 연관 키워드 크롤링

In [1]:
import requests 
from bs4 import BeautifulSoup
from urllib.request import urlopen

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd
from datetime import datetime
import time 
import re

### 1. 네이버 해시태그 크롤링

In [2]:
query = '따릉이+후기'
startdate = '20200101'
finishdate = '20201231'

url = "https://search.naver.com/search.naver?where=view&query=" + query + "&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom" + startdate + "to" + finishdate

print(url)

https://search.naver.com/search.naver?where=view&query=따릉이+후기&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20200101to20201231


In [4]:
driver = webdriver.Chrome() 

driver.get(url) # Get the page
print(driver.current_url)

https://search.naver.com/search.naver?where=view&query=%EB%94%B0%EB%A6%89%EC%9D%B4+%ED%9B%84%EA%B8%B0&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20200101to20201231


In [5]:
driver.get(url)
last_height = driver.execute_script("return document.body.scrollHeight") #스크롤의 크기를 확인

while True: 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤은 맨 아래로 내림
    time.sleep(3)
    new_height = driver.execute_script("return document.body.scrollHeight") #다시한번 스크롤의 크기를 확인
    print(new_height)
    print(last_height)

    if new_height == last_height: #처음에 확인한 스크롤의 크기와 맨 아래로 내린 후 확인한 스크롤의 크기를 비교
        break
    last_height = new_height

10943
5793
15854
10943
15854
15854


In [6]:
source = BeautifulSoup(driver.page_source, 'html.parser')

# 가상 브라우저 내 HTML 항목 모두 긁어옴

In [7]:
# 네이버 View 내 Hash-tag 추출

view_tags = source.find_all('a', {'class' : 'tag'}) # ResultSet (리스트와 유사한 형태)

tag_list = []

for tag in view_tags:
    tag_info = tag.get_text()
    tag_info = tag_info.replace('#', '') # '#'을 제거
    tag_list.append(tag_info)

df_tag_list = pd.DataFrame(tag_list)

df_tag_list.rename(columns = {0 : 'Title'}, inplace = True)

df_tag_list.to_csv('따릉이_hashtag_keywords.csv', encoding='UTF-8', index=False)
    
print(len(tag_list))


driver.close()
driver.quit()

453


### 2. 네이버 뉴스 크롤링

In [34]:
query = '따릉이'

start_date = '2019.01.01'
end_date = '2019.12.31'

max_page = 10

In [36]:
titles = []

start_date = start_date.replace(".", "")
end_date = end_date.replace(".", "")


current_call = 1
last_call = (max_page - 1) * 10 + 1 

while current_call <= last_call:
    
    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    url = "https://search.naver.com/search.naver?where=news&query=" + query \
          + "&nso=so%3Ar%2Cp%3Afrom" + start_date \
          + "to" + end_date \
          + "%2Ca%3A&start=" + str(current_call)

    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class' : "info"}):
        if urls["href"].startswith("https://news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')

            title = source_news.find('h3', {'id' : 'articleTitle'}).get_text()
            print('Processing article : {}'.format(title))

        
        
            titles.append(title)
          
        except:
            print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
            
   
    time.sleep(5)
    current_call += 10
    
    


1번째 기사글부터 크롤링을 시작합니다.
Processing article : 서울시민 60% "공유정책 서비스 이용해봤다"…'따릉이' 으뜸
Processing article : ‘따릉이’ 서울 공유정책 인지도 1위
Processing article : [르포] 제로페이 1년, 점주도 손님도 '생소하다'
Processing article : 3년 연속…서울시민이 뽑은 1위 정책은 ‘따릉이'
Processing article : [현장에선] 왜 ‘따릉이’밖에 없을까
Processing article : 서울시, 올해 공유정책 만족도 ‘장난감’이 가장 높아
Processing article : [서울 29초영화제 시상식] '따릉이' 타고 5분 레이스…독특한 설정으로 웃음 선사
Processing article : 따릉이, 공공와이파이, 돌봄SOS “올해 서울시 최고 정책”

11번째 기사글부터 크롤링을 시작합니다.
Processing article : 서울시 10대 뉴스 1위, 3년 연속 '따릉이'
Processing article : 4㎞내 사용자 71%...서울 '틈새 교통수단'된 따릉이
Processing article : 서울시 "따릉이 이용해 배달 엄격 금지" 경고
Processing article : '서울은 지금 따릉이 열풍'
Processing article : 한국IT전문강사협동조합, 노인 위한 '따릉이 이용법' 영상 제작
Processing article : 5등급車 못 다니는 서울 사대문 안, 버스신설·따릉이 2배 확대
Processing article : 시민 편의 위해 만들었더니…'따릉이 배달' 골머리
Processing article : 서울공공자전거 `따릉이` 운행중단…태풍 링링 북상 영향
Processing article : 서울시 "따릉이로 배달하지 마"…배달대행 업체 난감
Processing article : 유행처럼 번진 ‘따릉이 절도’…중고생 30여 명 입건

21번째 기사글부터 크롤링을 시작합니다.
Processing arti

In [37]:
naver_article_df = pd.DataFrame(titles)
                          
naver_article_df.rename(columns = {0 : 'Title'}, inplace = True)

naver_article_df.to_csv('따릉이_naver_keywords.csv', index=False, encoding='utf-8')
naver_article_df.head()

,Title
0,"서울시민 60% ""공유정책 서비스 이용해봤다""…'따릉이' 으뜸"
1,‘따릉이’ 서울 공유정책 인지도 1위
2,"[르포] 제로페이 1년, 점주도 손님도 '생소하다'"
3,3년 연속…서울시민이 뽑은 1위 정책은 ‘따릉이'
4,[현장에선] 왜 ‘따릉이’밖에 없을까


### 3. 구글 페이지 크롤링

In [12]:
max_page = 10
current_call = 0
last_call = (max_page - 1) * 10

In [14]:
titles = []

driver = webdriver.Chrome()   


while current_call <= last_call:
    
    print('\n{}번째 페이지부터 크롤링을 시작합니다.'.format(current_call))
    
    base = 'https://www.google.com/search?q=' 
    query = '따릉이코스'
    url = base + query + "&start=" +str(current_call)
    
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html)
    list_items = soup.find_all('div', {'class':'g'})

    for item in list_items:
        try:
            title = item.find('h3').get_text()           
             
            titles.append(title)     
            
        except:
            print('출력 불가 항목')
    
    time.sleep(5)
    current_call +=10
    
    
driver.close()  
driver.quit() 



0번째 페이지부터 크롤링을 시작합니다.

10번째 페이지부터 크롤링을 시작합니다.

20번째 페이지부터 크롤링을 시작합니다.

30번째 페이지부터 크롤링을 시작합니다.

40번째 페이지부터 크롤링을 시작합니다.

50번째 페이지부터 크롤링을 시작합니다.

60번째 페이지부터 크롤링을 시작합니다.

70번째 페이지부터 크롤링을 시작합니다.

80번째 페이지부터 크롤링을 시작합니다.

90번째 페이지부터 크롤링을 시작합니다.


In [15]:
google_article_df = pd.DataFrame(titles)

google_article_df.rename(columns = {0 : 'Title'}, inplace = True)

google_article_df.to_csv('따릉이_google_keywords.csv', index=False, encoding='utf-8')
google_article_df.head()   

,Title
0,서울자전거 따릉이를 타고 둘러보는 서울 투어 - 서울관광
1,가을 바람 따라~ 따릉이 타기 좋은 라이딩 코스 4곳 | 서울시 - 내 ...
2,[서울사랑] '따릉이'로 서울 여행! 추천코스 3선 | 서울시 정보소통 ...
3,따릉이로 달리기 좋은 자전거 코스 4 | 대학내일
4,서울 #따릉이 여행 코스 3 | 1boon


### 4. 유튜브 제목 크롤링

In [16]:
keyword = '따릉이 브이로그'
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)

In [17]:
driver = webdriver.Chrome() 
driver.implicitly_wait(1)   # 크롬 브라우저 내부 대기
driver.get(url)


body = driver.find_element_by_tag_name('body')   #'tag name'을 통해 element에 접근
num_of_pagedowns = 100               

while num_of_pagedowns:                         # 페이지 스크롤 - 스크롤 내려서 동영상 목록 가져오기
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

    
soup = BeautifulSoup(driver.page_source, 'html.parser')
youtube_title = soup.select('#video-title > yt-formatted-string')

driver.close()
driver.quit()

In [18]:
youtube_title_list = []

In [19]:
for tag in youtube_title:
    youtube_title_list.append(tag.get_text().strip())    

In [6]:
youtube_title_list

['(SUB) VLOGㅣ잔잔한 자취생 브이로그 (공부, 카누라떼, 따릉이, 비빔국수)',
 '( 구블로그#02 📹 ) 서울에서 따릉이 타고 논 VLOG(브이로그) / 따릉이 대여하는 법 / 따릉이 코스',
 'VLOG | 🚲한강에서 따릉이타는 브이로그  | 지은 Jieun',
 '[퇴근후 브이로그] 더키버니랑 함께, 따릉이 첫도전!',
 '[VLOG] 일상 브이로그 | 직장인 브이로그 | 언박싱｜늦잠지옥｜주말치팅(?)｜뮤지컬｜따릉이 늦바람｜cos｜보건옥｜잠수교집｜먹로그 (잠봉뵈르,타코야키,소고기해장국, bbq 핫윙)',
 '따릉이와 함께 한 퇴근길 브이로그',
 '한강 첫 따릉이 브이로그 (Feat.한강 치맥)',
 'Vlog. 4월 마지막 주 브이로그.따릉이타고 한강라이딩! 갈릭버터파스타도해먹고 여유를 조금씩 찾아 가보자',
 '직장인 브이로그 - 따릉이와 함께한 하루, 따릉이 출퇴근🚴\u200d♀️',
 '히부vlog. 직장인 브이로그, 따릉이와 한강 데이트, 일하는 평일과 노는 주말',
 '[vlog] 도시락 싸서 따릉이 타고 한강가기🚲. 홍제천에서 난지한강공원까지 씽씽 달려! 계란샌드위치, 자두잼, 딸기. 자매 브이로그. 일상 브이로그.',
 '따릉이로 서울 골목길 여행을 해보는 브이로그(feat.*킨 도넛)',
 '생각보다 심각한 상태인 요즘 아이돌들 무대수준ㄷㄷ(이건 아니지...)',
 'study with me • 밤에 집에서 같이 공부해요 • real time • 장작 타는 소리 • fire crackling',
 '1000만원짜리 자전거로 만원짜리 자전거 참교육하기 ㅋㅋㅋㅋㅋㅋ [ 그들의 삶 2탄 : 자전거편 with 권재관 ] 공대생 변승주',
 '[한강라이딩] 잔낙이 눈돌아 가요~~~',
 '따릉이 타고 가서 마트 장보기 브이로그',
 'Vlog. 뜽자매 브이로그-  예배. 한강. 따릉이. 젠틀몬스터. 곱창. 앨리웨이. 성수미술관. 드로잉. 새학기 준비. 고기리 막국수.',
 'VLOG I 한강 자전거 라이딩 브이로그 (feat. 따릉이, 

In [20]:
youtube_df = pd.DataFrame(youtube_title_list)
youtube_df.rename(columns = {0 : 'Title'}, inplace = True)

youtube_df.to_csv('따릉이_Youtube_keywords.csv', encoding='', index=False)

youtube_df.head()

,Title
0,( 구블로그#02 📹 ) 서울에서 따릉이 타고 논 VLOG(브이로그) / 따릉이 대...
1,"[퇴근후 브이로그] 더키버니랑 함께, 따릉이 첫도전!"
2,[VLOG] 일상 브이로그 | 직장인 브이로그 | 언박싱｜늦잠지옥｜주말치팅(?)｜뮤...
3,Vlog. 4월 마지막 주 브이로그.따릉이타고 한강라이딩! 갈릭버터파스타도해먹고 여...
4,"(SUB) VLOGㅣ잔잔한 자취생 브이로그 (공부, 카누라떼, 따릉이, 비빔국수)"


### 5. 브런치 크롤링

In [22]:
driver = webdriver.Chrome()

url = 'https://brunch.co.kr/search?q=%EB%94%B0%EB%A6%89%EC%9D%B4&type=article'
    
driver.get(url) 
print(driver.current_url)

https://brunch.co.kr/search


In [23]:
driver.get(url)

last_height = driver.execute_script("return document.body.scrollHeight")  

while True: 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # execute_script 함수 사용 (자바스크립트로 아이디, 패스워드를 넘겨줌)
      
    time.sleep(3)
    new_height = driver.execute_script("return document.body.scrollHeight") 
    print(last_height)

    if new_height == last_height: 
        break
    last_height = new_height

source = BeautifulSoup(driver.page_source, 'html.parser')         # 페이지의 elements모두 가져오기
brunch_tags = source.find_all('a', {'class' : 'tit_subject'}) 
tag_list = ['']
for tag in brunch_tags:
    tag_info = tag.get_text()
    tag_info = tag_info.replace('#', '')
    tag_list.append(tag_info)
print(tag_list)

driver.close()
driver.quit()

300
1068
1622
2213
2803
4181
7133
7723
8117
['']


In [24]:
source.find_all('strong', {'class':'tit_subject'})

[<strong class="tit_subject"><b>따릉이</b>가 가져다준 것 - 전에 보지 못하던 것이 보이기 시작했다.</strong>,
 <strong class="tit_subject">'<b>따릉이</b>' 적자 논란, '이름 붙이기'는 왜 위험한가! - 5월 5일 뉴스공장 인터뷰</strong>,
 <strong class="tit_subject"><b>따릉이</b>는 낭만적이지 못해 - 달을 쏘다.</strong>,
 <strong class="tit_subject">쓰레기 더미 <b>따릉이</b> 보관소 - 나팔꽃</strong>,
 <strong class="tit_subject"><b>따릉이</b>의 마법 - 페달을 밟는 시기</strong>,
 <strong class="tit_subject"><b>따릉이</b>로 자전거 배우기 - 서른 다섯, 드디어 자전거를 마스터하다. 그것도 독학으로!</strong>,
 <strong class="tit_subject"><b>따릉이</b>와 여유의 다섯 - 이번 가을은 <b>따릉이</b> 덕에 빛났다</strong>,
 <strong class="tit_subject">주말엔 <b>따릉이</b></strong>,
 <strong class="tit_subject">이사, 그리고 <b>따릉이</b> - 새 술은 새 부대에. 환경이 변하니 일상이 달라졌다.</strong>,
 <strong class="tit_subject">(2) <b>따릉이</b>를 타고 출근해보자 - 자전거 우선 도로엔 왜 자전거가 없죠?</strong>,
 <strong class="tit_subject"><b>따릉이</b>와 헬멧 - <b>따릉이</b> 라이딩 = 헬멧 필수</strong>,
 <strong class="tit_subject"><b>따릉이</b>를 타며 국가의 역할에 관해 묻다.</strong>,
 <strong class="tit_subject">9월 28일, 월요일 휴가 - 도망친 여자, <b>따릉이</

In [25]:
source.find_all('strong', {'class':'tit_subject'})[0].get_text()

'따릉이가 가져다준 것 - 전에 보지 못하던 것이 보이기 시작했다.'

In [26]:
brunch_subjects = source.find_all('strong', {'class' : 'tit_subject'})

subject_list = []

for subject in brunch_subjects:
    subject_list.append(subject.get_text())

print(subject_list)

['따릉이가 가져다준 것 - 전에 보지 못하던 것이 보이기 시작했다.', "'따릉이' 적자 논란, '이름 붙이기'는 왜 위험한가! - 5월 5일 뉴스공장 인터뷰", '따릉이는 낭만적이지 못해 - 달을 쏘다.', '쓰레기 더미 따릉이 보관소 - 나팔꽃', '따릉이의 마법 - 페달을 밟는 시기', '따릉이로 자전거 배우기 - 서른 다섯, 드디어 자전거를 마스터하다. 그것도 독학으로!', '따릉이와 여유의 다섯 - 이번 가을은 따릉이 덕에 빛났다', '주말엔 따릉이', '이사, 그리고 따릉이 - 새 술은 새 부대에. 환경이 변하니 일상이 달라졌다.', '(2) 따릉이를 타고 출근해보자 - 자전거 우선 도로엔 왜 자전거가 없죠?', '따릉이와 헬멧 - 따릉이 라이딩 = 헬멧 필수', '따릉이를 타며 국가의 역할에 관해 묻다.', '9월 28일, 월요일 휴가 - 도망친 여자, 따릉이, 그리고...', '이준석과 따릉이', '따릉이 출퇴근의 묘미 - 내가 따릉이로 출근, 퇴근하며 하는 것들', '따릉이 연간 회원권 - 든든한 나의 이동수단', '따릉이를 타고 도서관에 가는 걸 좋아합니다. - 이런 게 내 취향, 그리고 코로나', '따릉이도 괜찮은데?', '공유자전거 따릉이 예찬론 - 어디든 쉽고 빠르게 갈 수 있는 것이 주는 힘', '서울시의 자랑! 공공 자전거 서비스 ‘서울자전거 따릉이', '사소한 따릉이 매너 - 기왕이면 해주면 좋음', '한강, 따릉이, 잡념 - 서사적인 존재란 무엇인가', '자전거와 전동킥보드 - 천천히 가도 난 따릉이가 좋더라', '스물 하나, 다시 운전 연습 - 따릉이에서 부릉이로', '박원순 시장이 돌아가신 날 경찰서에 가다. - 박원순 서울 특별시장, 서울로 7017, 따릉이, 문화 비축기지, 원순씨', '따릉이 출퇴근을 하다. - 자전거를 타고 붕붕', '함께 타는 서울자전거! 따릉이', '착한 따릉이 사용법 - 따릉이 일일권 대여 정보', '서울시의 자전거 대여 서비스 따릉이를 아시나요?', '따릉이 완전초보를 위한 입문기 - 이동시간이

In [27]:
brunchDic = {'Title' : subject_list}
df_brunch = pd.DataFrame(brunchDic)
df_brunch.to_csv('따릉이_brunch_keywords.csv', encoding='', index=False)

df_brunch.head()

,Title
0,따릉이가 가져다준 것 - 전에 보지 못하던 것이 보이기 시작했다.
1,"'따릉이' 적자 논란, '이름 붙이기'는 왜 위험한가! - 5월 5일 뉴스공장 인터뷰"
2,따릉이는 낭만적이지 못해 - 달을 쏘다.
3,쓰레기 더미 따릉이 보관소 - 나팔꽃
4,따릉이의 마법 - 페달을 밟는 시기
